# Лабораторная работа 3. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы Курмашев Данил БПИ2303
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Джонсона



### Выполнение:

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog, filedialog
import math, random, time
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from heapq import heappush, heappop
from collections import deque

INF = float('inf')
graph = {}
last_path = []
last_path_cost = None

def parse_adjacency_matrix(text):
    """Парсинг матрицы смежности в граф"""
    text = text.strip()
    if not text:
        return None
    lines = text.splitlines()
    n = len(lines)
    graph_data = {}
    for i, line in enumerate(lines):
        parts = line.split()
        if len(parts) != n:
            return None
        graph_data[i] = {}
        for j, val in enumerate(parts):
            try:
                w = float(val)
            except:
                if val.lower() in ("inf", "infty", "∞"):
                    w = INF
                else:
                    return None
            if i == j:
                continue
            if w != 0 and w != INF:
                if math.isfinite(w) and abs(w - round(w)) < 1e-9:
                    w = int(round(w))
                graph_data[i][j] = w
    for i in range(n):
        graph_data.setdefault(i, {})
    return graph_data

def parse_incidence_matrix(text):
    """Парсинг матрицы инцидентности в граф"""
    text = text.strip()
    if not text:
        return None
    lines = text.splitlines()
    matrix = [line.split() for line in lines]
    n = len(matrix)
    if n == 0:
        return None
    m = len(matrix[0])
    for row in matrix:
        if len(row) != m:
            return None
    graph_data = {i: {} for i in range(n)}
    for j in range(m):
        edge_vertices = []
        direction = {}
        for i in range(n):
            try:
                val = int(matrix[i][j])
            except:
                val = 0
            if val != 0:
                edge_vertices.append(i)
                direction[i] = val
        if len(edge_vertices) < 2:
            continue
        u = edge_vertices[0]
        v = edge_vertices[1]
        val_u = direction[u]
        val_v = direction[v]
        if val_u == val_v:
            graph_data[u][v] = 1
            graph_data[v][u] = 1
        else:
            if val_u > 0 and val_v < 0:
                graph_data[u][v] = 1
            elif val_u < 0 and val_v > 0:
                graph_data[v][u] = 1
    return graph_data

def load_graph_action():
    """Загружает граф из файла (матрица смежности или инцидентности, пользователь выбирает тип)"""
    global graph
    filename = filedialog.askopenfilename(title="Загрузить граф", 
                                          filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])
    if not filename:
        return
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = f.read()
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось прочитать файл:\n{e}")
        return
    g = parse_adjacency_matrix(data)
    if g is None:
        g = parse_incidence_matrix(data)
    if g is None:
        messagebox.showerror("Ошибка", "Не удалось распознать формат графа. Требуется матрица смежности или инцидентности.")
        return
    graph = g
    clear_path()
    current_algo = notebook.tab(notebook.select(), "text")
    num_nodes = len(graph)
    num_edges = sum(len(neigh) for neigh in graph.values())
    result_texts[current_algo].insert(tk.END, f"Граф загружен. Вершин: {num_nodes}, Рёбер: {num_edges}.\n")
    result_texts[current_algo].see(tk.END)

def create_graph_action():
    """Открывает окно для генерации случайного графа"""
    global graph
    n = simpledialog.askinteger("Создать граф", "Введите число вершин:", parent=window, minvalue=1)
    if n is None:
        return
    m = simpledialog.askinteger("Создать граф", "Введите число рёбер:", parent=window, minvalue=0)
    if m is None:
        return
    max_edges = n * (n - 1)
    if m > max_edges:
        messagebox.showinfo("Инфо", f"Число рёбер ограничено {max_edges} для {n} вершин (без петель).")
        m = max_edges
    graph = {i: {} for i in range(n)}
    all_possible_edges = [(i, j) for i in range(n) for j in range(n) if i != j]
    random.shuffle(all_possible_edges)
    edges = all_possible_edges[:m]
    for u, v in edges:
        w = random.randint(1, 10)
        graph[u][v] = w
    clear_path()
    current_algo = notebook.tab(notebook.select(), "text")
    result_texts[current_algo].insert(tk.END, f"Случайный граф создан. Вершин: {n}, Рёбер: {m}.\n")
    result_texts[current_algo].see(tk.END)
    show_graph_action()

def get_shortest_path_edges(path):
    """Возвращает список рёбер, образующих кратчайший путь"""
    return [(path[i], path[i+1]) for i in range(len(path)-1)]

def clear_path():
    """Очистка пути к файлу"""
    global last_path, last_path_cost
    last_path = []
    last_path_cost = None

def dijkstra_algo(src, dest):
    """Алгоритм Дейкстры"""
    for u in graph:
        for w in graph[u].values():
            if w < 0:
                return None, None, "Граф содержит рёбра с отрицательным весом. Алгоритм Дейкстры неприменим."
    dist = {node: INF for node in graph}
    dist[src] = 0
    prev = {node: None for node in graph}
    heap = [(0, src)]
    while heap:
        d, u = heappop(heap)
        if d != dist[u]:
            continue
        if u == dest:
            break
        for v, w in graph[u].items():
            newd = d + w
            if newd < dist[v]:
                dist[v] = newd
                prev[v] = u
                heappush(heap, (newd, v))
    if dist.get(dest, INF) == INF:
        return INF, [], None
    path = []
    node = dest
    while node is not None:
        path.append(node)
        node = prev[node]
    path.reverse()
    return dist[dest], path, None

def bellman_ford_algo(src, dest):
    """Алгоритм Белламана-Форда"""
    dist = {node: INF for node in graph}
    dist[src] = 0
    prev = {node: None for node in graph}
    n = len(graph)
    for _ in range(n - 1):
        updated = False
        for u in graph:
            if dist[u] == INF:
                continue
            for v, w in graph[u].items():
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    prev[v] = u
                    updated = True
        if not updated:
            break
    for u in graph:
        if dist[u] == INF:
            continue
        for v, w in graph[u].items():
            if dist[u] + w < dist[v]:
                return None, None, "Граф содержит циклы отрицательного веса (достижимые из исходной вершины)."
    if dist.get(dest, INF) == INF:
        return INF, [], None
    path = []
    node = dest
    while node is not None:
        path.append(node)
        node = prev[node]
    path.reverse()
    return dist[dest], path, None

def levit_algo(src, dest):
    """Алгоритм Левита"""
    dist = {node: INF for node in graph}
    dist[src] = 0
    prev = {node: None for node in graph}
    state = {node: 2 for node in graph}
    state[src] = 1
    q = deque([src])
    count = {node: 0 for node in graph}
    count[src] = 1
    while q:
        u = q.popleft()
        state[u] = 0
        for v, w in graph[u].items():
            if dist[u] + w < dist[v]:
                if state[v] == 2:
                    dist[v] = dist[u] + w
                    prev[v] = u
                    state[v] = 1
                    q.append(v)
                    count[v] += 1
                    if count[v] > len(graph):
                        return None, None, "Граф содержит циклы отрицательного веса."
                elif state[v] == 1:
                    dist[v] = dist[u] + w
                    prev[v] = u
                    count[v] += 1
                    if count[v] > len(graph):
                        return None, None, "Граф содержит циклы отрицательного веса."
                else:
                    dist[v] = dist[u] + w
                    prev[v] = u
                    state[v] = 1
                    q.appendleft(v)
                    count[v] += 1
                    if count[v] > len(graph):
                        return None, None, "Граф содержит циклы отрицательного веса."
    if dist.get(dest, INF) == INF:
        return INF, [], None
    path = []
    node = dest
    while node is not None:
        path.append(node)
        node = prev[node]
    path.reverse()
    return dist[dest], path, None

def floyd_warshall_algo(src, dest):
    """Алгоритм Флойда-Уоршелла"""
    nodes = list(graph.keys())
    for u in list(nodes):
        for v in graph[u]:
            if v not in graph:
                graph[v] = {}
    nodes = list(graph.keys())
    dist = {i: {j: INF for j in nodes} for i in nodes}
    next_node = {i: {j: None for j in nodes} for i in nodes}
    for i in nodes:
        dist[i][i] = 0
        for j, w in graph[i].items():
            if w < dist[i][j]:
                dist[i][j] = w
                next_node[i][j] = j
    for k in nodes:
        for i in nodes:
            if dist[i][k] == INF:
                continue
            for j in nodes:
                if dist[k][j] == INF:
                    continue
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_node[i][j] = next_node[i][k]
    for u in nodes:
        if dist[u][u] < 0:
            return None, None, "Граф содержит циклы отрицательного веса."
    if src not in graph or dest not in graph or dist[src][dest] == INF:
        return INF, [], None
    path = [src]
    u = src
    while u != dest:
        u = next_node[u][dest]
        if u is None:
            return INF, [], None
        path.append(u)
    return dist[src][dest], path, None

def johnson_algo(src, dest):
    """Алгоритм Джонсона"""
    q = "__temp_source__"
    while q in graph:
        q += "_"
    graph[q] = {node: 0 for node in graph.keys()}
    dist_q = {node: INF for node in graph}
    dist_q[q] = 0
    for _ in range(len(graph) - 1):
        updated = False
        for u in graph:
            if dist_q[u] == INF:
                continue
            for v, w in graph[u].items():
                if dist_q[u] + w < dist_q[v]:
                    dist_q[v] = dist_q[u] + w
                    updated = True
        if not updated:
            break
    for u in graph:
        if dist_q[u] == INF:
            continue
        for v, w in graph[u].items():
            if dist_q[u] + w < dist_q[v]:
                graph.pop(q, None)
                return None, None, "Граф содержит циклы отрицательного веса."
    h = dist_q
    new_graph = {}
    for u in graph:
        if u == q:
            continue
        new_graph[u] = {}
        for v, w in graph[u].items():
            if v not in h or u not in h or h[u] == INF or h[v] == INF:
                continue
            new_graph[u][v] = w + h[u] - h[v]
    graph.pop(q, None)
    if src not in new_graph:
        if src == dest:
            return 0, [src], None
        else:
            return INF, [], None
    dist_rew = {node: INF for node in new_graph}
    dist_rew[src] = 0
    prev = {node: None for node in new_graph}
    heap = [(0, src)]
    while heap:
        d, u = heappop(heap)
        if d != dist_rew[u]:
            continue
        if u == dest:
            break
        for v, w in new_graph[u].items():
            newd = d + w
            if newd < dist_rew[v]:
                dist_rew[v] = newd
                prev[v] = u
                heappush(heap, (newd, v))
    if dest not in dist_rew or dist_rew[dest] == INF:
        return INF, [], None
    path = []
    node = dest
    while node is not None:
        path.append(node)
        node = prev[node]
    path.reverse()
    total_cost = 0
    for i in range(len(path) - 1):
        u = path[i]; v = path[i+1]
        total_cost += graph[u][v]
    return total_cost, path, None

def yen_algo(src, dest, K=3):
    """Алгоритм Йена"""
    for u in graph:
        for w in graph[u].values():
            if w < 0:
                return None, None, "Граф содержит рёбра с отрицательным весом. Алгоритм Йена неприменим."
    dist = {node: INF for node in graph}
    prev = {node: None for node in graph}
    dist[src] = 0
    heap = [(0, src)]
    while heap:
        d, u = heappop(heap)
        if d != dist[u]:
            continue
        if u == dest:
            break
        for v, w in graph[u].items():
            newd = d + w
            if newd < dist[v]:
                dist[v] = newd
                prev[v] = u
                heappush(heap, (newd, v))
    if dist.get(dest, INF) == INF:
        return INF, [], None
    path0 = []
    node = dest
    while node is not None:
        path0.append(node)
        node = prev[node]
    path0.reverse()
    cost0 = dist[dest]
    if K <= 1:
        return cost0, path0, None
    A = [(path0, cost0)]
    B = []
    import copy
    for k in range(1, K):
        if k > len(A):
            break
        prev_path, prev_cost = A[k-1]
        n = len(prev_path)
        for i in range(n - 1):
            spur_node = prev_path[i]
            root_path = prev_path[:i+1]
            root_cost = 0
            for j in range(len(root_path) - 1):
                u = root_path[j]; v = root_path[j+1]
                root_cost += graph[u][v]
            temp_graph = copy.deepcopy(graph)
            root_excluded = set(root_path[:-1])
            for rn in root_excluded:
                temp_graph.pop(rn, None)
            for u in list(temp_graph.keys()):
                for rn in root_excluded:
                    temp_graph[u].pop(rn, None)
            if spur_node in temp_graph and i+1 < len(prev_path):
                next_node = prev_path[i+1]
                temp_graph[spur_node].pop(next_node, None)
            dist_spur = {node: INF for node in temp_graph}
            prev_spur = {node: None for node in temp_graph}
            if spur_node not in temp_graph:
                continue
            dist_spur[spur_node] = 0
            heap2 = [(0, spur_node)]
            while heap2:
                d, u = heappop(heap2)
                if d != dist_spur[u]:
                    continue
                if u == dest:
                    break
                for v, w in temp_graph[u].items():
                    newd = d + w
                    if newd < dist_spur[v]:
                        dist_spur[v] = newd
                        prev_spur[v] = u
                        heappush(heap2, (newd, v))
            if dist_spur.get(dest, INF) == INF:
                continue
            spur_path = []
            node2 = dest
            while node2 is not None:
                spur_path.append(node2)
                node2 = prev_spur[node2]
            spur_path.reverse()
            if not spur_path or spur_path[0] != spur_node:
                continue
            combined_path = root_path + spur_path[1:]
            spur_cost = dist_spur[dest]
            total_cost = root_cost + spur_cost
            skip = False
            for p, c in A:
                if p == combined_path:
                    skip = True
                    break
            if skip:
                continue
            for p, c in B:
                if p == combined_path:
                    skip = True
                    break
            if skip:
                continue
            B.append((combined_path, total_cost))
        if not B:
            break
        B.sort(key=lambda x: x[1])
        best_path, best_cost = B.pop(0)
        A.append((best_path, best_cost))
    return A, None, None

def show_graph_action():
    """Показать граф"""
    if not graph:
        messagebox.showerror("Ошибка", "Граф не загружен. Сначала загрузите или создайте граф.")
        return
    plt.close('all')
    G = nx.DiGraph()
    for u in graph:
        for v, w in graph[u].items():
            G.add_edge(u, v, weight=w)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(6, 4))
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, edgelist=list(G.edges()), arrows=True, arrowstyle='->', arrowsize=15, edge_color='gray')
    if last_path:
        path_edges = get_shortest_path_edges(last_path)
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, arrows=True, arrowstyle='->', arrowsize=15, edge_color='red', width=2)
    edge_labels = {(u, v): data['weight'] for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.axis('off')
    plt.title("Визуализация графа")
    plt.show(block=False)

def find_shortest_path_action(algo):
    """Поиск кратчайшего пути"""

    global last_path, last_path_cost

    if not graph:
        messagebox.showerror("Ошибка", "Граф не задан. Загрузите или создайте граф.")
        return

    try:
        src = int(src_vars[algo].get())
    except:
        src = src_vars[algo].get().strip()
        if not src:
            messagebox.showerror("Ошибка ввода", "Не указана начальная вершина.")
            return

    try:
        dest = int(dest_vars[algo].get())
    except:
        dest = dest_vars[algo].get().strip()
        if not dest:
            messagebox.showerror("Ошибка ввода", "Не указана конечная вершина.")
            return

    if src not in graph or dest not in graph:
        messagebox.showerror("Ошибка ввода", "Начальная или конечная вершина не найдена в графе.")
        return

    text_widget = result_texts[algo]
    text_widget.delete("1.0", tk.END)
    last_path = []
    last_path_cost = None
    error = None
    cost, path = None, []

    algo_functions = {
        "Dijkstra": lambda s, d: dijkstra_algo(s, d),
        "Bellman-Ford": lambda s, d: bellman_ford_algo(s, d),
        "Levit": lambda s, d: levit_algo(s, d),
        "Floyd-Warshall": lambda s, d: floyd_warshall_algo(s, d),
        "Johnson": lambda s, d: johnson_algo(s, d),
        "Yen": lambda s, d: yen_algo(s, d, 3)
    }

    if algo not in algo_functions:
        text_widget.insert(tk.END, "Выберите вкладку с конкретным алгоритмом для поиска пути.\n")
        return

    if algo == "Yen":
        result, path_error, error = algo_functions[algo](src, dest)
        if error is None:
            if isinstance(result, list) and result:
                last_path = result[0][0]
                last_path_cost = result[0][1]
                text_widget.insert(tk.END, f"{len(result)} кратчайших путей от {src} до {dest}:\n")
                for idx, (p, c) in enumerate(result, start=1):
                    text_widget.insert(tk.END, f"{idx}) Путь: {p}, длина: {c}\n")
                text_widget.see(tk.END)
                show_graph_action()
                return
            else:
                cost, path = result if isinstance(result, tuple) else (None, None)
        else:
            cost, path = None, None
    else:
        cost, path, error = algo_functions[algo](src, dest)

    if error:
        text_widget.insert(tk.END, error + "\n")
    elif cost == INF or not path:
        text_widget.insert(tk.END, f"Путь от {src} до {dest} не существует.\n")
    else:
        last_path = path
        last_path_cost = cost
        text_widget.insert(tk.END, f"Кратчайший путь от {src} до {dest}: {path} (длина: {cost})\n")

    text_widget.see(tk.END)
    show_graph_action()

def analyze_complexity_action(algo):
    """Анализ времени выполнения алгоритма(ов)"""

    global graph

    if not graph:
        messagebox.showerror("Ошибка", "Граф не задан. Загрузите или создайте граф.")
        return

    text_widget = result_texts[algo]
    text_widget.delete("1.0", tk.END)

    nodes_range = list(range(5, min(50, len(graph) + 10), 5)) if len(graph) > 5 else list(range(2, 10))

    algos_to_test = ["Floyd-Warshall", "Bellman-Ford", "Dijkstra", "Levit", "Johnson", "Yen"] if algo == "Comparison" else [algo]
    results = {a: [] for a in algos_to_test}

    for num_nodes in nodes_range:
        # Генерация случайного графа для тестов
        test_graph = {chr(65 + i): {} for i in range(num_nodes)}
        nodes = list(test_graph.keys())
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j and np.random.rand() < 0.3:
                    test_graph[nodes[i]][nodes[j]] = np.random.randint(1, 10)

        graph = test_graph
        src, dest = nodes[0], nodes[-1] if len(nodes) > 1 else nodes[0]

        for a in algos_to_test:
            start_time = time.time()

            if a == "Floyd-Warshall":
                _ = floyd_warshall_algo(src, dest)
            elif a == "Bellman-Ford":
                _ = bellman_ford_algo(src, dest)
            elif a == "Dijkstra":
                _ = dijkstra_algo(src, dest)
            elif a == "Levit":
                _ = levit_algo(src, dest)
            elif a == "Johnson":
                _ = johnson_algo(src, dest)
            elif a == "Yen":
                _ = yen_algo(src, dest, 3)

            elapsed_time = (time.time() - start_time) * 1000.0
            results[a].append(elapsed_time)

    plt.close('all')
    plt.figure(figsize=(8, 6) if algo == "Comparison" else (6, 4))

    for algo_name, times in results.items():
        plt.plot(nodes_range, times, marker='o', label=algo_name, linewidth=2 if algo != "Comparison" else 1)

    plt.title("Сравнение времени работы алгоритмов" if algo == "Comparison" else f"Анализ сложности алгоритма {algo}")
    plt.xlabel("Количество вершин")
    plt.ylabel("Время (мс)")
    plt.grid(True)
    if algo == "Comparison":
        plt.legend()
    plt.tight_layout()
    plt.show(block=False)

    if algo == "Comparison":
        text_widget.insert(tk.END, "Время выполнения алгоритмов:\n")
        for a in results:
            text_widget.insert(tk.END, f"{a}: {['{:.2f}'.format(t) for t in results[a]]} мс\n")
    else:
        text_widget.insert(tk.END, f"Время выполнения {algo}:\n")
        for count, time_val in zip(nodes_range, results[algo]):
            text_widget.insert(tk.END, f"{count} вершин: {time_val:.2f} мс\n")

    text_widget.see(tk.END)


window = tk.Tk()
window.title("Визуализация алгоритмов на графах")
window.geometry("1000x600")

notebook = ttk.Notebook(window)
notebook.pack(fill='both', expand=True)

algorithms = ["Floyd-Warshall", "Bellman-Ford", "Dijkstra", "Johnson", "Levit", "Yen", "Comparison"]
frames = {}
src_vars = {}
dest_vars = {}
result_texts = {}

for algo in algorithms:
    frame = ttk.Frame(notebook)
    notebook.add(frame, text=algo)
    frames[algo] = frame

    # Поля ввода начальной и конечной вершин
    src_vars[algo] = tk.StringVar()
    dest_vars[algo] = tk.StringVar()
    ttk.Label(frame, text="Начало:").grid(row=0, column=0, padx=5, pady=5, sticky='e')
    src_entry = ttk.Entry(frame, textvariable=src_vars[algo], width=10)
    src_entry.grid(row=0, column=1, padx=5, pady=5, sticky='w')
    ttk.Label(frame, text="Конец:").grid(row=0, column=2, padx=5, pady=5, sticky='e')
    dest_entry = ttk.Entry(frame, textvariable=dest_vars[algo], width=10)
    dest_entry.grid(row=0, column=3, padx=5, pady=5, sticky='w')

    # Кнопки управления
    load_btn = ttk.Button(frame, text="Загрузить граф", command=load_graph_action)
    load_btn.grid(row=1, column=0, padx=5, pady=5)
    create_btn = ttk.Button(frame, text="Создать граф", command=create_graph_action)
    create_btn.grid(row=1, column=1, padx=5, pady=5)
    show_btn = ttk.Button(frame, text="Показать граф", command=show_graph_action)
    show_btn.grid(row=1, column=2, padx=5, pady=5)
    find_btn = ttk.Button(frame, text="Найти путь", command=lambda a=algo: find_shortest_path_action(a))
    find_btn.grid(row=2, column=0, padx=5, pady=5)
    analyze_btn = ttk.Button(frame, text="Анализ сложности", command=lambda a=algo: analyze_complexity_action(a))
    analyze_btn.grid(row=2, column=1, padx=5, pady=5)

    # Текстовое поле для вывода результатов
    text_area = tk.Text(frame, wrap='word')
    text_area.grid(row=3, column=0, columnspan=4, padx=5, pady=5, sticky='nsew')
    frame.rowconfigure(3, weight=1)
    frame.columnconfigure(3, weight=1)
    result_texts[algo] = text_area

def on_tab_change(event):
    """Закрытие дочерних окон при переходе на другую вкладку"""
    plt.close('all')

notebook.bind("<<NotebookTabChanged>>", on_tab_change)

window.mainloop()


ModuleNotFoundError: No module named 'matplotlib'